# Reguläre Ausdrücke

[Videos](https://www.youtube.com/playlist?list=PLWeMgMhRDsIFk8RPO6C3DzcQfBf8mTGn7)

Reguläre Ausdrücke sind sehr mächtig, aber eher schwierig zu lesen. Sie sind eine Art "kompaktes Programmieren mit Zeichen". Sie bieten die Möglichkeit, in einem String auf clevere Art zu suchen und Teilstrings zu extrahieren oder zu ersetzen.

In [ ]:
a     passt auf a
.     passt auf jedes Zeichen

Ein Match liegt auch vor, wenn ein Teilstring zu dem regulären Ausdruck passt.

In [2]:
import re
a = re.search('.a','ba')
print(a)
bool(a)      

<_sre.SRE_Match object; span=(0, 2), match='ba'>


True

In [ ]:
bool(re.search('.a','bb'))

In [ ]:
bool(re.search('.a','cca'))

### Zeichenklassen

In [ ]:
[0-5]         Ziffern von 0 bis 5
[a-zA-Z0-9]   englische Buchstaben und Ziffern

### Vordefinierte Zeichenklassen

In [ ]:
\d   Ziffer, entspricht [0-9]
\w   Buchstaben der meisten Sprachen, der Unterstrich und Ziffern
\s   whitespace (blank, tabs etc.)
\D, \W, \S  Negation der entsprechenden Klassen

In [ ]:
a = ['4ab9', '0ab1', '6ab9', '0bb5']
regex = '[0-5]ab\d'
for text in a:
    print(bool(re.search(regex,text)))

### Quantoren
Quantoren beziehen sich auf das/die vorangegangene Zeichen/Zeichenklasse.

In [ ]:
?     null-  oder einmal
*     beliebig oft (auch null)
+     mindestens einmal
{k}   genau k-mal
{,k}  höchstens k-mal
{k,}  mindestens k-mal
{k,l} zwischen k- und l-mal

In [12]:
a = ['8acDa 9','8abcd_9','7xxx 9','0_a6b 9']
regex = '\d\w{4}\s9'
for text in a:
    print(bool(re.search(regex,text)))

True
False
False
True


### Sonstige Zeichen

In [ ]:
|           Alternative
^ oder \A   Anfang eines Strings
$ oder \Z   Ende eines Strings
\b, \B      Anfang, Ende eines Worts
(....)      Gruppe

In [18]:
a = ['04','10' ,'99', '110' ,'112']
regex = '[1-9][0-9]'
for text in a:
    print(bool(re.search(regex,text)))

False
True
True
True
True


In [19]:
a = ['04' ,'10', '99', '110' ,'112']
regex = '^[1-9][0-9]$'                 
for text in a:
    print(bool(re.search(regex,text)))

False
True
True
False
False


In [21]:
a = ['99','100' ,'299', '310']
regex = '^1[0-9][0-9]|2[0-9][0-9]$'    
for text in a:
    print(bool(re.search(regex,text)))

False
True
True
False


In [22]:
a = ['-3.14', '2','+000.62','4.-62']
regex = '^[-+]?[0-9]*\.?[0-9]+$'     
for text in a:
    print(bool(re.search(regex,text)))

True
True
True
False


## Verwendung

Um zu verhindern, dass das \ als normales Escape-Zeichen gelesen wird, setzen wir vor den regex-String ein r. Damit wird z.B. \b nicht als backspace, sondern als Wortgrenze interpretiert. Nach wie vor können wir die speziellen Zeichen der regulären Ausdrücke mit \ escapen.

In [23]:
a = ['ccb.a', 'ccb.ad', 'bab', 'b.a?c']
regex = r'b\.a.'
for text in a:
    print(bool(re.search(regex,text)))

False
True
False
True


### re.findall()
Alle Treffer in eine Liste sammeln

In [25]:
s = 'abe 12 aa 43 4ab 113 01 99 5j'
regex = r'\b[1-9][0-9]\b'
a = re.findall(regex,s)
print(a)

['12', '43', '99']


Daten extrahieren, hier sollen jeweils die Vornamen gesammelt werden.

In [27]:
a = ['Nachname = Schulte, Vorname = Anna-Lena, Alter = 21',
     'Vorname= Malte, Nachname= Riedberg,  Alter = 22']
regex = 'Vorname\s*=\s*([A-Za-z-]+)'
for s in a:
    print(re.findall(regex,s)[0])

Anna-Lena
Malte


### re.sub()
Jeder Treffer im letzten String, wird durch den mittleren ersetzt.

In [28]:
regex = r'[Jj]a[Vv]a'
print(re.sub(regex,'Python', 'Java oder java und jaVa'))

Python oder Python und Python


### re.split()
Strings aufspalten. Alles was matched, ist Separator.

In [29]:
 
regex = r'[,; ]'      
print(re.split(regex,'a-a;bbb,,c; dd'))
regex = r'[,; ]+'     # + ist greedy
print(re.split(regex,'a-a;bbb,,c; dd'))

['a-a', 'bbb', '', 'c', '', 'dd']
['a-a', 'bbb', 'c', 'dd']


### Gruppe ( ... )
Die Gruppenklammern kennzeichnen, was extrahiert wird. Mit (?:....) gruppiert man, ohne dass die Gruppe extrahiert wird.

In [32]:
text = 'abab665  ab91 ababab42  abc809'
regex = r'\b(ab)+(\d+)'
print(re.findall(regex,text))


[('ab', '665'), ('ab', '91'), ('ab', '42')]


In [34]:
text = 'abab665  ab91 ababab42  abc809'
regex = r'\b(?:ab)+(\d+)'
print(re.findall(regex,text))

['665', '91', '42']


### Greedy
Die Quantoren ? * + sind standardmäßig greedy, d.h. sie suchen beim matching den größtmöglichen String. Mit einem angefügten ? werden die Quantoren genügsam und suchen nach dem kleinstmöglich passenden Teilstring.

In [36]:
print(re.findall('Py.*on','Python Python Python'))
print(re.findall('Py.*?on','Python Python Python'))

['Python Python Python']
['Python', 'Python', 'Python']


### Anwendungsbeispiel
Bei der Umstellung von Processing-Programmen nach P5JS muss z.B. folgende Transformation vorgenommen werden:
<br/>
`... fill(#AB02F9); ...  ->  ... fill('#AB02F9'); ...`

In [4]:
zeile = 'fill(#AB02F9);  // Farbe setzen '
zeile = re.sub(r"fill\((#[A-Fa-f0-9]{6})\);", r"fill('\1');", zeile)
print(zeile)

fill('#AB02F9');  // Farbe setzen 
